In [1]:
# Working with the data
import requests
import io
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
from sklearn.preprocessing import LabelEncoder
from io import BytesIO
from zipfile import ZipFile
import os
import tempfile

# Visualisation
import matplotlib.pylab as plt
import matplotlib.patches as mpatches
import folium
from folium import plugins
import seaborn as sns
import branca.colormap as cm
from branca.colormap import StepColormap

%matplotlib inline

In [2]:
# Define a function to download and read a CSV file.
def download_and_load_csv(url):
    response = requests.get(url)
    csv_string = response.content.decode('utf-8')
    df = pd.read_csv(io.StringIO(csv_string))
    return df

# API Link
download_link_1 = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/business-establishments-with-address-and-industry-classification/exports/csv?lang=en&timezone=Australia%2FSydney&use_labels=true&delimiter=%2C'
download_link_2 = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/property-level-energy-consumption-modelled-on-building-attributes-baseline-2011-/exports/csv?lang=en&timezone=Australia%2FSydney&use_labels=true&delimiter=%2C'

# Use functions to download and load data
location_df = download_and_load_csv(download_link_1)
energy_df = download_and_load_csv(download_link_2)

In [3]:
location_df

,Census year,Block ID,Property ID,Base property ID,CLUE small area,Trading name,Business address,Industry (ANZSIC4) code,Industry (ANZSIC4) description,Longitude,Latitude
0,2004,44,108110,108110,Melbourne (CBD),Total Tel International,"Part Level 12, 140-0 Queen Street MELBOURNE 3000",5809,Other Telecommunications Services,144.961185,-37.815391
1,2004,44,108110,108110,Melbourne (CBD),CJ Ham & Murray,"Part Floor 2, 140 Queen Street MELBOURNE 3000",6720,Real Estate Services,144.961185,-37.815391
2,2004,44,108111,108111,Melbourne (CBD),Netherlands Consulate,"Suite 7, Part Level 4, 118-126 Queen Street ME...",7552,Foreign Government Representation,144.961308,-37.815656
3,2004,44,108111,108111,Melbourne (CBD),Thomas Koutsoupias,"Suite 10, Level 9, 118-126 Queen Street MELBOU...",6931,Legal Services,144.961308,-37.815656
4,2004,44,108111,108111,Melbourne (CBD),K S T Partners,"Suite 4, Floor 5, 118 Queen Street MELBOURNE 3000",6932,Accounting Services,144.961308,-37.815656
...,...,...,...,...,...,...,...,...,...,...,...
374205,2017,266,106082,106082,Carlton,RMIT University (BLD 42) (Ericson Building),36-40 Lygon Street CARLTON 3053,8102,Higher Education,144.965375,-37.805471
374206,2017,266,106082,106082,Carlton,RMIT University (BLD 95),24-26 Lygon Street CARLTON 3053,8102,Higher Education,144.965375,-37.805471
374207,2017,266,107083,107083,Carlton,RMIT University,11-13 Orr Street CARLTON 3053,8102,Higher Education,144.965017,-37.806389
374208,2017,266,107087,107087,Carlton,Vacant,8-14 Orr Street CARLTON 3053,0,Vacant Space,144.965370,-37.806513


In [4]:
energy_df

,Geo Point,Geo Shape,total_2011,p_com_2011,p_res_2026,p_com_2026,total_2026,p_com_2021,property_id,p_res_2011,total_2016,total_2021,p_res_2021,floor_area,p_res_2016,p_com_2016
0,"-37.803829913238744, 144.94499251543567","{""coordinates"": [[[[144.94514309252298, -37.80...",5.913673,0.000000,34.439604,0.000000,34.439604,0.000000,100011.0,5.913673,26.342193,30.220318,30.220318,255.0,26.342193,0.000000
1,"-37.8037392383391, 144.9450094179755","{""coordinates"": [[[[144.9451597400275, -37.803...",54.057413,54.057404,0.000000,59.886065,59.886065,58.044081,100012.0,0.000000,56.140379,58.044081,0.000000,0.0,0.000000,56.140379
2,"-37.80344814421259, 144.9450614676207","{""coordinates"": [[[[144.94520809645564, -37.80...",3.817222,0.000000,22.230427,0.000000,22.230427,0.000000,100016.0,3.817222,17.003627,19.506919,19.506919,164.6,17.003627,0.000000
3,"-37.80330160793391, 144.9450879361556","{""coordinates"": [[[[144.94524107836415, -37.80...",66.479567,66.479558,0.000000,73.647619,73.647619,71.382355,100018.0,0.000000,69.041191,71.382355,0.000000,0.0,0.000000,69.041191
4,"-37.80313100914576, 144.94511862327707","{""coordinates"": [[[[144.94526549030562, -37.80...",3.543570,0.000000,20.636751,0.000000,20.636751,0.000000,100021.0,3.543570,15.784655,18.108488,18.108488,152.8,15.784655,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13582,"-37.82599534624159, 144.93826784707588","{""coordinates"": [[[[144.9383830100469, -37.825...",149.805000,149.805000,0.000000,144.532800,144.532800,146.258883,0.0,0.000000,148.096434,146.258883,0.000000,0.0,0.000000,148.096434
13583,"-37.8287231458785, 144.92255617927117","{""coordinates"": [[[[144.92247078961668, -37.82...",165.317000,165.317000,0.000000,159.501600,159.501600,161.403420,0.0,0.000000,163.431242,161.403420,0.000000,0.0,0.000000,163.431242
13584,"-37.82920659916404, 144.9216073579147","{""coordinates"": [[[[144.9213798102216, -37.829...",67.572000,67.572000,0.000000,65.192400,65.192400,65.972092,0.0,0.000000,66.800945,65.972092,0.000000,0.0,0.000000,66.800945
13585,"-37.82602780261689, 144.94091595936325","{""coordinates"": [[[[144.9411644299442, -37.826...",11.447000,11.447000,0.000000,19.674000,19.674000,17.089412,0.0,0.000000,14.410492,17.089412,0.000000,0.0,0.000000,14.410492


In [ ]:
import requests
import pandas as pd
import io

# Define the function to download and read a CSV file
def download_and_load_csv(url):
    response = requests.get(url)
    csv_string = response.content.decode('utf-8')
    df = pd.read_csv(io.StringIO(csv_string))
    return df

# Download links
download_link_1 = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/business-establishments-with-address-and-industry-classification/exports/csv?lang=en&timezone=Australia%2FSydney&use_labels=true&delimiter=%2C'
download_link_2 = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/property-level-energy-consumption-modelled-on-building-attributes-baseline-2011-/exports/csv?lang=en&timezone=Australia%2FSydney&use_labels=true&delimiter=%2C'

# Use the function to download and load data
location_df = download_and_load_csv(download_link_1)
energy_df = download_and_load_csv(download_link_2)

# Assuming 'Year', 'Industry (ANZSIC4)', 'Longitude', and 'Latitude' are columns in location_df
# Group the data by Industry Code, Longitude, and Latitude
grouped_data = location_df.groupby(['Industry (ANZSIC4) code	', 'Longitude', 'Latitude'])

# Filter data for the years 2021 and 2011
data_2021 = location_df[location_df['Census year'] == 2021]
data_2011 = location_df[location_df['Census year'] == 2011]

# Apply grouping to these subsets
grouped_2021 = data_2021.groupby(['Industry (ANZSIC4)', 'Longitude', 'Latitude'])
grouped_2011 = data_2011.groupby(['Industry (ANZSIC4)', 'Longitude', 'Latitude'])




In [ ]:
grouped_2021

In [ ]:
grouped_2011